<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [6]:
## Import libraries
import gym
import numpy as np
from time import time
import matplotlib.pyplot as plt
%matplotlib inline
import torch
from collections import namedtuple, deque

Our current state is represented by a 210x160x3 Tensor. This represents the height, length, and the three RGB color channels of the Atari game.

In [7]:
## Initiate the environment
env = gym.make("MsPacman-v0")

print('State shape is {}\n'.format(env.observation_space.shape))
print('Possible actions are {}: '.format(env.action_space.n))
print(env.env.get_action_meanings())

State shape is (210, 160, 3)

Possible actions are 9: 
['NOOP', 'UP', 'RIGHT', 'LEFT', 'DOWN', 'UPRIGHT', 'UPLEFT', 'DOWNRIGHT', 'DOWNLEFT']


In [8]:
# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
## Create a class that stores the agent's experiences and retrieves random batches from its memory
class ReplayMemory:
    
    def __init__(self, capacity=10000, batch_size=32):
        """Initiate the class parameters"""
        self.capacity = capacity
        self.batch_size = batch_size
        self.buffer = deque(maxlen=capacity)
        self.experience = namedtuple('Transition', ('state', 'action', 'next_state', 'reward', 'done'))

    def push(self, state, action, reward, next_state, done):
        """Add a new experience to the memory buffer"""
        e = self.experience(state, action, reward, next_state, done)
        self.buffer.append(e)

    def __len__(self):
        return len(self.buffer)        
        
    def sample(self):
        """Sample a random batch of experiences from the memory"""
        samples = random.sample(self.buffer, self.batch_size)

        states = np.array([state[0] for state in samples])
        actions = np.array([action[1] for action in samples])
        rewards = np.array([reward[2] for reward in samples])
        next_states = np.array([next_state[3] for next_state in samples])
        dones = np.array([done[4] for done in samples])
        
        return (states, actions, rewards, next_states, dones)
    


In [10]:
import random
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D
from keras.models import Sequential
from keras.optimizers import Adam

class DQNAgent:

    def __init__(self, observation_space, action_space, capacity=10000, batch_size=32, 
                 learning_rate=0.001, gamma = 0.95, epsilon=1.0, epsilon_min = 0.05, epsilon_decay = 0.9999):
        """Creates an instance of a Deep-Q-Network Agent"""
        self.state_shape = observation_space.shape # shape of any given state
        self.action_size = action_space.n # number of possible actions
        self.learning_rate = learning_rate # model's learning rate
        self.gamma = gamma # discount rate
        self.batch_size = batch_size # size of batches sampled from memory
        self.epsilon = epsilon  # exploration rate
        self.epsilon_min = epsilon_min # threshold of epsilon
        self.epsilon_decay = epsilon_decay # decay of exploration rate
        self.model = self.build_model() # instance of dqn model
        self.memory = ReplayMemory(capacity=capacity, batch_size=self.batch_size) # expirience replay memory

    def build_model(self):
        """Build the Convolutional DQN model"""
        model = Sequential()
        # add convolutional layers
        model.add(Conv2D(32, (3, 3), activation='relu', input_shape=self.state_shape))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(Flatten())
        # add fully connected layers with dropout
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(32, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(16, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(self.action_size, activation='linear'))
        # compile model with MSE loss and Adam optimizer
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def get_action(self, state):
        """Select action according to the epsilon-greedy policy"""
        # get the correct dimension that the models accepts
        # transform state from shape (210,160,3) to (1,210,160,3)
        state = np.expand_dims(state, axis=0) 
        action = self.model.predict(state)
        if random.random() > self.epsilon:
            return np.argmax(action) # exploitation
        else:
            return random.choice(np.arange(self.action_size)) # exploration

    def train_model(self, states, actions, rewards, next_states, dones):
        """Train the model using random batches of experiences from the memory"""
        states, actions, rewards, next_states, dones = self.memory.sample() # random memory sample
        # iterate over the batch
        for i in range(len(states)):
            state, action, reward, next_state, done = states[i], actions[i], rewards[i], next_states[i], dones[i]

            state = np.expand_dims(state, axis=0)
            next_state = np.expand_dims(next_state, axis=0)

            target = self.model.predict(state)
            target_val = self.model.predict(next_state)

            if done:
                target[0][action] = reward
            else:
                target[0][action] = (reward + self.gamma * np.max(target))
            
            # get the loss of the model
            loss = self.model.fit(state, target, verbose=0).history['loss'][0]
            return loss

Using TensorFlow backend.


In [11]:
EPISODES = 500 # Number of episodes
losses = []

# Get an instance of a DQN agent
dqnAgent = DQNAgent(env.observation_space, env.action_space, capacity=10000, batch_size=32, 
                 learning_rate=0.001, gamma = 0.95, epsilon=1.0, epsilon_min = 0.05, epsilon_decay = 0.9999)

In [12]:
## Train the agent
print('Training initiated...')
ep_rewards = []
for episode in range(1, EPISODES+1):
    init_time = time()
    state = env.reset()
    ep_reward = 0
    done=False
    while not done:
        action = dqnAgent.get_action(state)
        next_state, reward, done, info = env.step(action)
        dqnAgent.memory.push(state, action, reward, next_state, done)
        ep_reward += reward
        state = next_state

        if dqnAgent.epsilon >= dqnAgent.epsilon_min: 
            dqnAgent.epsilon *= dqnAgent.epsilon_decay # decaying exploration/exploitation
    time_taken = time() - init_time
    loss = dqnAgent.train_model(action, state, next_state, reward, done)
    losses.append(loss)
    ep_rewards.append(ep_reward)
    if episode % 10 == 0:
        print('Episode {} of {}:\n   reward: {:.2f} \n   loss: {:.2f} \n   \u03B5: {:.3f} \n   time taken: {:.2f} sec'.format(episode, EPISODES, ep_reward, loss, dqnAgent.epsilon, time_taken))
        
print('Training ended.')

Training initiated...


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv2d_1/convolution (defined at c:\python38\lib\site-packages\keras\backend\tensorflow_backend.py:3007) ]] [Op:__inference_keras_scratch_graph_355]

Function call stack:
keras_scratch_graph


In [ ]:
plt.plot(losses)
plt.yscale('log')
plt.show()

In [ ]:
min(losses)
